# Initial Model Building

In [87]:
import pandas as pd
import numpy as np
import datetime
import pickle 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, roc_auc_score
import seaborn as sns

from sklearn.metrics import f1_score
from sklearn import metrics
import sklearn.metrics as metrics

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [43]:
df = pd.read_csv('/Users/markdetiberiis/Desktop/Patient_no_show/cleaned_data.csv')

In [47]:
#Awaiting days
le = LabelEncoder()
df['waiting_days'] = le.fit_transform(df['waiting_days'])

# #times
# le = LabelEncoder()
# df['schedule_time'] = le.fit_transform(df['schedule_time'])
# le = LabelEncoder()
# df['appointment_time'] = le.fit_transform(df['appointment_time'])

# #dates

# le = LabelEncoder()
# df['appointment_date'] = le.fit_transform(df['appointment_date'])


In [48]:
df.drop(['schedule_time'],axis = 1, inplace = True)
df.drop(['appointment_date'],axis = 1, inplace = True)
df.drop(['appointment_time'],axis = 1, inplace = True)

In [49]:
df.drop(['Unnamed: 0'],axis = 1, inplace = True)

In [50]:
df.drop(['PatientId'],axis = 1, inplace = True)

In [51]:
df.drop(['AppointmentID'],axis = 1, inplace = True)

In [52]:
# Get the Dependent and Independent Features.
X = df.drop(['NoShow'], axis=1)
y = df['NoShow']

In [53]:
#test train split 80/20

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [54]:
#fit scaler

scaler = StandardScaler()  
scaler.fit(X_train)

X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test) 

In [55]:
#initialize decesion tree

clf = DecisionTreeClassifier()

In [56]:
#fit to training data 

clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [57]:
#show feature importance

for col, importance in zip(X.columns, np.sort(clf.feature_importances_)[::-1]):
    print("{} -- {:.2f}".format(col, importance))

Gender -- 0.24
Age -- 0.23
Neighbourhood -- 0.17
Scholarship -- 0.07
Hypertension -- 0.07
Diabetes -- 0.05
Alcoholism -- 0.04
Handicap -- 0.04
SMSReceived -- 0.02
appointment_weekday -- 0.02
schedule_weekday -- 0.01
waiting_days -- 0.01
schedule_date_Y -- 0.01
schedule_date_M -- 0.01
schedule_date_D -- 0.01
appointment_date_Y -- 0.01
appointment_date_M -- 0.00
appointment_date_D -- 0.00


In [135]:
clf.score(X_test, y_test)

0.7286257124762507

In [31]:
#Initia;ize and fit random forest

rfc = RandomForestClassifier(class_weight='balanced')

rfc.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced')

In [32]:
#importances

for col, importance in zip(X.columns, np.sort(rfc.feature_importances_)[::-1]):
    print("{} -- {:.2f}".format(col, importance))
    

Gender -- 0.27
Age -- 0.23
Neighbourhood -- 0.19
Scholarship -- 0.06
Hypertension -- 0.05
Diabetes -- 0.04
Alcoholism -- 0.03
Handicap -- 0.03
SMSReceived -- 0.02
appointment_weekday -- 0.02
schedule_weekday -- 0.01
waiting_days -- 0.01
schedule_date_Y -- 0.01
schedule_date_M -- 0.01
schedule_date_D -- 0.01
appointment_date_Y -- 0.01
appointment_date_M -- 0.00
appointment_date_D -- 0.00


In [33]:
rfc.score(X_test, y_test)

0.7872975662716005

In [34]:
y_pred = rfc.predict(X_train)
print(f1_score(y_pred, y_train))

y_pred = rfc.predict(X_test)
print(f1_score(y_pred, y_test))

0.980727342909923
0.27861307149432335


In [21]:
#set params dict

params={'n_estimators':[10,20,25,30], 'max_depth': range(1,20,1), 'min_samples_split':[2,3,4,5]}


In [35]:
#Use GridSearch

grid_tree = GridSearchCV(rfc, params, cv=5, n_jobs=-1, verbose=1)
grid_tree.fit(X, y)
print('Best Params') 
print(grid_tree.best_params_)

print('Best Score') 
print(grid_tree.best_score_)

Fitting 5 folds for each of 304 candidates, totalling 1520 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   21.4s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done 1520 out of 1520 | elapsed: 10.6min finished


Best Params
{'max_depth': 19, 'min_samples_split': 2, 'n_estimators': 30}
Best Score
0.7287363111743497


In [61]:
metrics.accuracy_score(y_pred, y_test)

0.7404776983624355

In [62]:
metrics.f1_score(y_pred, y_test)

0.12103569787038454

In [65]:
filename = 'rfc_model.pk1'

pickle.dump(grid_tree, open(filename, 'wb'))


In [66]:
scaler_filename = 'rfc_scaler.pk1'

pickle.dump(scaler, open(scaler_filename, 'wb'))


In [ ]:
df.to_csv('/Users/markdetiberiis/Desktop/Patient_no_show/model_ready_data.csv')

In [76]:
#create df for each DOW for later predictions by day

df_Monday = df[df['appointment_weekday']==0]
df_Tuesday = df[df['appointment_weekday']==1]
df_Wednesday = df[df['appointment_weekday']==2]
df_Thursday = df[df['appointment_weekday']==3]
df_Friday = df[df['appointment_weekday']==4]

In [78]:
df_Monday.to_csv('/Users/markdetiberiis/Desktop/Patient_no_show/monday_appts')
df_Tuesday.to_csv('/Users/markdetiberiis/Desktop/Patient_no_show/tuesday_appts')
df_Wednesday.to_csv('/Users/markdetiberiis/Desktop/Patient_no_show/wednesday_appts')
df_Thursday.to_csv('/Users/markdetiberiis/Desktop/Patient_no_show/thursday_appts')
df_Friday.to_csv('/Users/markdetiberiis/Desktop/Patient_no_show/friday_appts')


In [122]:
corr_NoShow = df.groupby(['NoShow']).corr(method = 'spearman', min_periods = 1)
corr_NoShow

Gender       Age  Neighbourhood  Scholarship  \
NoShow                                                                       
0      Gender               1.000000 -0.103158      -0.007422    -0.111676   
       Age                 -0.103158  1.000000      -0.015814    -0.093697   
       Neighbourhood       -0.007422 -0.015814       1.000000     0.005428   
       Scholarship         -0.111676 -0.093697       0.005428     1.000000   
       Hypertension        -0.057806  0.501534       0.006991    -0.017443   
       Diabetes            -0.032893  0.292007       0.015634    -0.024610   
       Alcoholism           0.114280  0.099796      -0.003964     0.028755   
       Handicap             0.023871  0.079651       0.009260    -0.006029   
       SMSReceived         -0.042044  0.032623       0.013624    -0.007880   
       appointment_weekday -0.009156  0.002370       0.006326     0.001383   
       schedule_weekday    -0.011795 -0.014640       0.011159     0.007492   
       waiting_days        -0.043374  0.102548      -0.008903    -0.035497   
       schedule_date_Y      0.006557 -0.023325      -0.016908     0.000067   
       schedule_date_M      0.013645 -0.017263       0.002714     0.022453   
       schedule_date_D      0.005494 -0.010076       0.006253     0.000965   
       appointment_date_Y        NaN       NaN            NaN          NaN   
       appointment_date_M  -0.008854  0.012590       0.001397     0.000120   
       appointment_date_D   0.000363 -0.009308       0.001560     0.000097   
1      Gender               1.000000 -0.124950      -0.009521    -0.123841   
       Age                 -0.124950  1.000000      -0.015686    -0.068564   
       Neighbourhood       -0.009521 -0.015686       1.000000    -0.010699   
       Scholarship         -0.123841 -0.068564      -0.010699     1.000000   
       Hypertension        -0.047889  0.502789      -0.000604    -0.023824   
       Diabetes            -0.031517  0.295811      -0.003187    -0.024142   
       Alcoholism           0.073959  0.112687       0.013189     0.057870   
       Handicap             0.016263  0.071572       0.018210    -0.019952   
       SMSReceived         -0.061197 -0.009479       0.037965     0.015221   
       appointment_weekday -0.000389 -0.002741      -0.003614    -0.001788   
       schedule_weekday    -0.015638 -0.023875       0.014186     0.014730   
       waiting_days        -0.041596  0.042916       0.015753    -0.013942   
       schedule_date_Y     -0.004598 -0.035429      -0.023418     0.005742   
       schedule_date_M      0.018052  0.024118       0.005283     0.006379   
       schedule_date_D     -0.004468  0.001555      -0.004942     0.022619   
       appointment_date_Y        NaN       NaN            NaN          NaN   
       appointment_date_M   0.005264  0.014900       0.006351    -0.010011   
       appointment_date_D  -0.002516 -0.008959       0.001225     0.005802   

                            Hypertension  Diabetes  Alcoholism  Handicap  \
NoShow                                                                     
0      Gender                  -0.057806 -0.032893    0.114280  0.023871   
       Age                      0.501534  0.292007    0.099796  0.079651   
       Neighbourhood            0.006991  0.015634   -0.003964  0.009260   
       Scholarship             -0.017443 -0.024610    0.028755 -0.006029   
       Hypertension             1.000000  0.431505    0.087519  0.084517   
       Diabetes                 0.431505  1.000000    0.018907  0.062192   
       Alcoholism               0.087519  0.018907    1.000000  0.005655   
       Handicap                 0.084517  0.062192    0.005655  1.000000   
       SMSReceived              0.004488 -0.008154   -0.030750 -0.023297   
       appointment_weekday      0.000378 -0.001225   -0.002989 -0.006520   
       schedule_weekday        -0.004908 -0.000866   -0.007947 -0.003986   
       waiting_days             0.044081  0.012562   -0.042852 -0.017676   
 

In [123]:
#calculate correlation between Hypertension and NoShow
column_1 = df['NoShow']
column_2 = df["Hypertension"]
correlation = column_1. corr(column_2) 

print(correlation)

-0.035701177345014996


In [128]:
#calculate correlation between Diabetes and NoShow
column_1 = df['NoShow']
column_2 = df["Diabetes"]
correlation = column_1. corr(column_2) 

print(correlation)

-0.01517994382077367


In [126]:
#calculate correlation between SMSReceived and NoShow
column_1 = df['NoShow']
column_2 = df["SMSReceived"]
correlation = column_1. corr(column_2) 

print(correlation)

0.12643065757314462


In [129]:
#calculate correlation between Age and NoShow

column_1 = df['NoShow']
column_2 = df["Age"]
correlation = column_1. corr(column_2) 

print(correlation)

-0.06031851140917367


In [130]:
#calculate correlation between Neighborhood and NoShow

column_1 = df['NoShow']
column_2 = df["Neighbourhood"]
correlation = column_1. corr(column_2) 

print(correlation)

-0.0089615130415546
